In [1]:
# until the github pip foolbox is updated
import sys
sys.path.insert(0,'/media/rene/code/foolbox')
import foolbox

In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))

from models.cifar import ResNet
from models import DenoiseNet, DenoiseHGD, DenoiseLoss, UNet

from utils.loading import load_net_cifar
from utils.data import make_gen_std_cifar, make_generators_DF_cifar
from utils.train_val import validate_epoch
from utils.train_val_denoise import validate_epoc_denoise
from utils.evaluation import evaluate_adv_files_df, get_metrics



import torch
from torch import nn
from torch.autograd import Variable
from torch.autograd.gradcheck import zero_gradients
import torchvision.transforms as T
from torchvision.models import resnet18, vgg16
import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(torch.cuda.device_count())
print(torch.cuda.current_device())

2
0


# ResNet50: Non-Adversarial Accuracy 

In [12]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'

with torch.cuda.device(1):
    model = load_net_cifar(model_loc).to(device)
    dataloaders = make_gen_std_cifar(PATH, batch_size, num_workers)
    validate_epoch(dataloaders['val'], model, criterion, device)

Loading model_file resnet-50
Files already downloaded and verified
Files already downloaded and verified
VALID:  * TOP1 94.020 TOP5 99.800 Loss (0.2266)	 Time (0.017)	


# ResNet50: Adversarial Accuracy 

In [16]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/sample/files_df_adv.pkl'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
            

with torch.cuda.device(1):
    model = load_net_cifar(model_loc).to(device)
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                           path_colname='path', adv_path_colname=None, return_loc=False)
    validate_epoch(dataloaders['val'], model, criterion, device)

Loading model_file resnet-50
VALID:  * TOP1 93.266 TOP5 99.799 Loss (0.2728)	 Time (0.043)	


In [17]:
with torch.cuda.device(1):
    model = load_net_cifar(model_loc).to(device)
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                           path_colname='adv_path', adv_path_colname=None, return_loc=False)
    validate_epoch(dataloaders['val'], model, criterion, device)

Loading model_file resnet-50
VALID:  * TOP1 34.874 TOP5 99.799 Loss (1.7130)	 Time (0.045)	


# Denoised ResNet50: Accuracy 

In [24]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)


with torch.cuda.device(1):
    denoiser = UNet(3, 3, stochastic=False)
    classifier = ResNet(50, num_classes=10, IM_SIZE=32)
    loss = DenoiseLoss(n=1, hard_mining=0, norm=False)
    
    model = DenoiseNet(classifer=classifier, denoiser=denoiser, loss=loss).to(device)
    model.load_state_dict(torch.load(denoise_net_loc)['state_dict'])  
    
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                           path_colname='path', adv_path_colname='adv_path', return_loc=False)
    
    validate_epoc_denoise(dataloaders['val'], model, requires_control=True, device=device)

Initializing Unet with stochastic:  False


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Original Acc 92.362 	 Adversarial Acc 85.829 Loss (189.3078)	Time (0.139)	 Data Time (0.028   Control Loss 158.878


# Denoised ResNet50: Adversarial Accuracy 

In [25]:
with torch.cuda.device(1):
    denoiser = UNet(3, 3, stochastic=False)
    classifier = ResNet(50, num_classes=10, IM_SIZE=32)
    loss = DenoiseLoss(n=1, hard_mining=0, norm=False)
    
    model = DenoiseNet(classifer=classifier, denoiser=denoiser, loss=loss).to(device)
    model.load_state_dict(torch.load(denoise_net_loc)['state_dict'])  
    
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                           path_colname='adv_path', adv_path_colname='path', return_loc=False)
    
    validate_epoc_denoise(dataloaders['val'], model, requires_control=True, device=device)

Initializing Unet with stochastic:  False


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Original Acc 45.628 	 Adversarial Acc 88.744 Loss (268.1241)	Time (0.138)	 Data Time (0.028   Control Loss 164.540


# Proper Evaluation

In [2]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
# with torch.cuda.device(1):

denoiser = UNet(3, 3, stochastic=False)
classifier = ResNet(50, num_classes=10, IM_SIZE=32)
loss = DenoiseLoss(n=1, hard_mining=0, norm=False)

denoise_model = DenoiseNet(classifer=classifier, denoiser=denoiser, loss=loss).to(device)
denoise_model.load_state_dict(torch.load(denoise_net_loc)['state_dict'])  

results = evaluate_adv_files_df(files_df, classifier, denoiser, denoise_model, device)

Initializing Unet with stochastic:  False


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [9]:
get_metrics(results)

Total Observations: 995
Original Accuracy: 92.36%
Adversarial Accuracy: 45.63%
Denoised Adversarial Accuracy : 85.83%
Percent of correct remaing correct: 90.32%


In [10]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_stochastic_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
with torch.cuda.device(1):
    denoiser = UNet(3, 3, stochastic=True)
    classifier = ResNet(50, num_classes=10, IM_SIZE=32)
    loss = DenoiseLoss(n=1, hard_mining=0, norm=False)

    denoise_model = DenoiseNet(classifer=classifier, denoiser=denoiser, loss=loss).to(device)
    denoise_model.load_state_dict(torch.load(denoise_net_loc)['state_dict'])  

    results = evaluate_adv_files_df(files_df, classifier, denoiser, denoise_model, device)
    
get_metrics(results)

Initializing Unet with stochastic:  True


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Total Observations: 995
Original Accuracy: 92.76%
Adversarial Accuracy: 47.74%
Denoised Adversarial Accuracy : 88.94%
Percent of correct remaing correct: 94.15%


# Evaluating PGD

In [6]:
batch_size = 128
num_workers = 4
criterion = nn.CrossEntropyLoss()
PATH = Path('/media/rene/data/')
model_loc = '/media/rene/data/adv_denoising/cifar10/cifar10_normal/models/resnet-50_model_best.pth.tar'
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_pgd/resnet50/sample/files_df_adv.pkl'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
            

with torch.cuda.device(1):
    model = load_net_cifar(model_loc).to(device)
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=32, 
                                           path_colname='adv_path', adv_path_colname=None, return_loc=False)
    validate_epoch(dataloaders['val'], model, criterion, device)

Loading model_file resnet-50
VALID:  * TOP1 40.800 TOP5 99.800 Loss (1.8857)	 Time (0.038)	


### Using Denoiser:

In [11]:
batch_size = 128
num_workers = 4
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
with torch.cuda.device(1):
    denoise_model = load_net_cifar(denoise_net_loc).to(device).eval()
    results = evaluate_adv_files_df(files_df, denoise_model, device)
    get_metrics(results)

Loading model_file unet
Initializing Unet with stochastic:  False


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Total Observations: 995
Original Accuracy: 92.36%
Adversarial Accuracy: 45.63%
Denoised Adversarial Accuracy : 85.83%
Percent of correct remaing correct: 90.32%


## FGSM attacked denoising UNet
* When you attack a model using a denoiser, removing the denoiser actually imporves perfromance.
* Is this jsut that any change in model makes the attack worse, i.e. transferability <100%

Should use load_net_cifar to load, and then get rid of classifier, etc. in evaluate_adv_files_df?

In [3]:
batch_size = 128
num_workers = 4
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50_denoised/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
with torch.cuda.device(1):
    denoise_model = load_net_cifar(denoise_net_loc).to(device).eval()
    results = evaluate_adv_files_df(files_df, denoise_model, device)
    get_metrics(results)

Loading model_file unet
Initializing Unet with stochastic:  False


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Total Observations: 997
Original Accuracy: 92.38%
Adversarial Accuracy: 85.76%
Denoised Adversarial Accuracy : 29.99%
Percent of correct remaing correct: 32.36%


### FGSM attacked stochastic denoised unet

In [10]:
batch_size = 128
num_workers = 4
files_df_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50_stoch_denoised/sample/files_df_adv.pkl'
denoise_net_loc = '/media/rene/data/adv_denoising/cifar10/adv_fgsm/resnet50/models/unet_stochastic_model_best.pth.tar'

with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
    
with torch.cuda.device(1):
    denoise_model = load_net_cifar(denoise_net_loc).to(device).eval()
    results = evaluate_adv_files_df(files_df, denoise_model, device)
    get_metrics(results)

Loading model_file unet
Initializing Unet with stochastic:  True


/media/rene/ADV/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


Total Observations: 993
Original Accuracy: 92.75%
Adversarial Accuracy: 76.94%
Denoised Adversarial Accuracy : 29.71%
Percent of correct remaing correct: 31.92%
